In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from scipy.io import wavfile
from scipy.signal import stft
from pathlib import Path

2024-03-29 06:31:11.709346: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 06:31:11.709444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 06:31:11.850402: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def load_and_process_wav(file_path):
    sample_rate, data = wavfile.read(file_path)
    
    freq, times, stft_data = stft(data, fs=sample_rate, nperseg=400, noverlap=200)  
    # Normalize the magnitude of STFT
    new_stft = np.abs(stft_data)
    new_stft /= np.max(new_stft)
    return new_stft

In [3]:
paths = [
    r"/kaggle/input/class-cv-lungs/ICBHI/COPD",
    r"/kaggle/input/class-cv-lungs/ICBHI/Healthy",
    r"/kaggle/input/class-cv-lungs/ICBHI/Pneumonia"
]

In [4]:
X_spectrograms = []
labels = []


for path in paths:
    base_path = Path(path)
    
    
    if path == r"/kaggle/input/class-cv-lungs/ICBHI/COPD":
        label = 'COPD'
    elif path == r"/kaggle/input/class-cv-lungs/ICBHI/Healthy":
        label = 'Healthy'
    elif path == r"/kaggle/input/class-cv-lungs/ICBHI/Pneumonia":
        label = 'Pneumonia'
    
    
    for wav_file_path in base_path.glob("*.wav"):
        # Load and process .wav file
        spectrogram = load_and_process_wav(wav_file_path)
        X_spectrograms.append(spectrogram)
        labels.append(label)

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)
y_categorical = to_categorical(y_encoded)

In [6]:
y_encoded

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
X_train, X_temp, y_train, y_temp = train_test_split(X_spectrograms, y_categorical, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [8]:
print(X_train.shape,)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
spectrogram.shape

In [ ]:

X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)



In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train[0].shape)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(np.array(X_train), np.array(y_train), epochs=30, batch_size=32, validation_data=(np.array(X_val), np.array(y_val)))
test_loss, test_acc = model.evaluate(np.array(X_test), np.array(y_test))
print(test_acc)


# C1

In [ ]:
    import matplotlib.pyplot as plt

    # Assuming the necessary imports and definitions for stft and wavfile.read are done

    file_path = "/kaggle/input/class-cv-lungs/ICBHI/COPD/107_2b3_Ar_mc_AKGC417L_1.wav"
    sample_rate, data = wavfile.read(file_path)

    freq, times, stft_data = stft(data, fs=sample_rate, nperseg=400, noverlap=200)  # Adjust nperseg and noverlap as needed
    # print(type(stft_data))
    print(stft_data.dtype)
    plotData = np.abs(stft_data.real).astype(int)
    # Normalize the magnitude of STFT
    new_stft = np.abs(stft_data)
    new_stft /= np.max(new_stft)
    x = new_stft[0:70,:]
#     Plot the spectrogram
    plt.figure(figsize=(10, 6))
    plt.imshow(x, aspect='auto', origin='lower', cmap='viridis')
    plt.colorbar(label='Magnitude')
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.title('COPD Spectrogram')

    plt.show()


In [ ]:
    import matplotlib.pyplot as plt

    # Assuming the necessary imports and definitions for stft and wavfile.read are done

    file_path = "/kaggle/input/class-cv-lungs/ICBHI/Healthy/144_1b1_Tc_sc_Meditron_2.wav"
    sample_rate, data = wavfile.read(file_path)

    freq, times, stft_data = stft(data, fs=sample_rate, nperseg=400, noverlap=100)  # Adjust nperseg and noverlap as needed
    # print(type(stft_data))
#     print(stft_data.dtype)

    plotData = np.abs(stft_data.real).astype(int)
#     plotData = stft_data.astype(int)

    # Normalize the magnitude of STFT
    new_stft = np.abs(stft_data)
    new_stft /= np.max(new_stft)
    x = new_stft[0:70,:]
#     Plot the spectrogram
    plt.figure(figsize=(10, 6))
    plt.imshow(x, aspect='auto', origin='lower', cmap='viridis')
    plt.colorbar(label='Magnitude')
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.title('HEALTHY Spectrogram')
    plt.show()


In [ ]:
x[60,20]

In [ ]:
    import matplotlib.pyplot as plt

    # Assuming the necessary imports and definitions for stft and wavfile.read are done

    file_path = "/kaggle/input/class-cv-lungs/ICBHI/Pneumonia/122_2b2_Tc_mc_LittC2SE_2.wav"
    sample_rate, data = wavfile.read(file_path)

    freq, times, stft_data = stft(data, fs=sample_rate, nperseg=400, noverlap=100)  # Adjust nperseg and noverlap as needed
    # print(type(stft_data))
#     print(stft_data.dtype)


#     plotData = stft_data.astype(int)

    # Normalize the magnitude of STFT
    new_stft = np.abs(stft_data)
    new_stft /= np.max(new_stft)
    x = new_stft[0:70,:]
#     Plot the spectrogram
    plt.figure(figsize=(10, 6))
    plt.imshow(x, aspect='auto', origin='lower', cmap='viridis')
    plt.colorbar(label='Magnitude')
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.title('PNEUMONIA Spectrogram')
    plt.show()


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from scipy.io import wavfile
from scipy.signal import stft
from pathlib import Path
from scipy.signal import resample


def load_and_process_wav(file_path):
    sample_rate, data = wavfile.read(file_path)
#     print(sample_rate)
    new_sample_rate = 3500
    resampled_data = resample(data, int(len(data) * (new_sample_rate / sample_rate)))

    freq, times, stft_data = stft(resampled_data, fs=sample_rate, nperseg=400, noverlap=200)  
#     Normalize the magnitude of STFT
    new_stft = np.abs(stft_data)
    new_stft /= np.max(new_stft)
    return new_stft


paths = [
    r"/kaggle/input/class-cv-lungs/ICBHI/COPD",
    r"/kaggle/input/class-cv-lungs/ICBHI/Healthy",
    r"/kaggle/input/class-cv-lungs/ICBHI/Pneumonia"
]


X_spectrograms = []
labels = []


for path in paths:
    base_path = Path(path)
    
    
    if path == r"/kaggle/input/class-cv-lungs/ICBHI/COPD":
        label = 'COPD'
    elif path == r"/kaggle/input/class-cv-lungs/ICBHI/Healthy":
        label = 'Healthy'
    elif path == r"/kaggle/input/class-cv-lungs/ICBHI/Pneumonia":
        label = 'Pneumonia'
    
    
    for wav_file_path in base_path.glob("*.wav"):

        spectrogram = load_and_process_wav(wav_file_path)
        X_spectrograms.append(spectrogram)
        labels.append(label)

        
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)
y_categorical = to_categorical(y_encoded)


X_train, X_temp, y_train, y_temp = train_test_split(X_spectrograms, y_categorical, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# instead of giving the dims of input as ( input shape , 1 ) in the cnn model directly, 
# we will add another auxillary dimension in the array itself

X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train[0].shape)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit(np.array(X_train), np.array(y_train), epochs=30, batch_size=32, validation_data=(np.array(X_val), np.array(y_val)))
test_loss, test_acc = model.evaluate(np.array(X_test), np.array(y_test))
print(test_acc)



# C1 with tuning

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from scipy.io import wavfile
from scipy.signal import stft
from pathlib import Path
import keras_tuner


import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD,Adam,Nadam
from keras import layers
from keras_tuner import RandomSearch,GridSearch,BayesianOptimization
# extra
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense,AveragePooling1D


# Load sound file
# sample_rate, data = wav.read('/kaggle/input/copd-class-cv/COPD/106_2b1_Pl_mc_LittC2SE_1.wav')

In [ ]:
def load_and_process_wav(file_path):
    sample_rate, data = wavfile.read(file_path)
    
    freq, times, stft_data = stft(data, fs=sample_rate, nperseg=400, noverlap=200)  
    # Normalize the magnitude of STFT
    new_stft = np.abs(stft_data)
    new_stft /= np.max(new_stft)
    return new_stft

In [ ]:
paths = [
    r"/kaggle/input/class-cv-lungs/ICBHI/COPD",
    r"/kaggle/input/class-cv-lungs/ICBHI/Healthy",
    r"/kaggle/input/class-cv-lungs/ICBHI/Pneumonia"
]

In [ ]:
X_spectrograms = []
labels = []


for path in paths:
    base_path = Path(path)
    
    
    if path == r"/kaggle/input/class-cv-lungs/ICBHI/COPD":
        label = 'COPD'
    elif path == r"/kaggle/input/class-cv-lungs/ICBHI/Healthy":
        label = 'Healthy'
    elif path == r"/kaggle/input/class-cv-lungs/ICBHI/Pneumonia":
        label = 'Pneumonia'
    
    
    for wav_file_path in base_path.glob("*.wav"):
        # Load and process .wav file
        spectrogram = load_and_process_wav(wav_file_path)
        X_spectrograms.append(spectrogram)
        labels.append(label)

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)
y_categorical = to_categorical(y_encoded)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X_spectrograms, y_categorical, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [ ]:
def build_model(hp):
    hp_neurons = hp.Int('neurons', min_value=32, max_value=512, step=64)
    hp_filters = hp.Int('filters', min_value=32, max_value=512, step=64)

    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train[0].shape)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))

#   Tune learning rate and batch size
    hp_learning_rate = hp.Choice('learning_rate', values=[0.01,0.0025, 0.001, 0.015])
    hp_batch_size = hp.Choice('batch_size', values=[2,4,16,32,64])
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

    return model

In [ ]:
tuner = RandomSearch(
                         build_model,
                         objective='val_accuracy',
                         max_trials=3,
                         executions_per_trial=2,
                         directory='',
                         project_name='heartbeat_tuning_random'
                )


hp_batch_size = tuner.oracle.get_space()['batch_size']
tuner.search(tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_categorical), epochs=5, validation_data=(X_test, encoded_test), batch_size=hp_batch_size)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split


def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train[0].shape)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))

    
#     hp_learning_rate = hp.Choice('learning_rate', values=[0.01,0.0025, 0.001, 0.015])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_batch_size = hp.Choice('batch_size', values=[2,4,16,32,64])
     
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model




X_train, X_temp, y_train, y_temp = train_test_split(X_spectrograms, y_categorical, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
 
tuner = RandomSearch(
  build_model,
  objective='val_accuracy',
  max_trials=1,   
  executions_per_trial=1,
  directory='my_dir',
  project_name='cnn_tuning')

 
tuner.search(tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train), epochs=10, validation_data=(X_val, y_val))

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

# Assuming you have X_spectrograms and y_categorical already defined

def build_model(hp):
    model = keras.Sequential()

    # Ensure X_train is a NumPy array
    X_train = np.asarray(X_train)

    # Validate input shape (optional)
    if X_train.shape[-1] < 1:
        raise ValueError("X_train must have at least 1 channel (grayscale) or 3 channels (RGB)")

    # Tune the number of filters for the first Conv2D layer
    hp_filters1 = hp.Int('filters1', min_value=16, max_value=64, step=16)
    model.add(layers.Conv2D(hp_filters1, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
    model.add(layers.MaxPooling2D((2, 2)))

    # Tune the number of filters for the second Conv2D layer
    hp_filters2 = hp.Int('filters2', min_value=32, max_value=128, step=32)
    model.add(layers.Conv2D(hp_filters2, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Tune the number of filters for the third Conv2D layer
    hp_filters3 = hp.Int('filters3', min_value=64, max_value=256, step=64)
    model.add(layers.Conv2D(hp_filters3, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())

    # Tune the number of units for the first Dense layer
    hp_units1 = hp.Int('units1', min_value=64, max_value=256, step=64)
    model.add(layers.Dense(hp_units1, activation='relu'))

    # Add a final Dense layer with the number of classes (assuming 3 classes)
    model.add(layers.Dense(3, activation='softmax'))  # 3 for 3 classes

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

    return model

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_spectrograms, y_categorical, test_size=0.2, random_state=42)

# No need to expand dimensions here (already done when defining X_train)

# Define the tuner with more trials and executions for a more robust search
tuner = RandomSearch(
  build_model,
  objective='val_accuracy',
  max_trials=10,  # Increased trials for better hyperparameter exploration
  executions_per_trial=3,  # Increased executions for more reliable results
  directory='my_dir',
  project_name='cnn_tuning')

# Start the hyperparameter search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model with early stopping (consider adding for potential improvement)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)

